In [1]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Common_Functions as cf 
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras import models 
import os
from keras.regularizers import l2
plt.rc('font', family='serif')
import matplotlib
matplotlib.rcParams.update({'font.size': 12})
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import chirallag as cL

In [2]:
def gen_and_load(n_pred,n_val):
    number_predictions= n_pred
    N = 3
    F0 = 1
    gens = cL.gen_gellman(3)
    pi=np.random.rand(number_predictions,N*N-1)
    dpi=np.random.rand(number_predictions,N*N-1)
    orig_V = cL.get_V(pi,dpi,gens,F0).real
    output = np.hstack((pi,dpi,np.expand_dims(orig_V,axis=1)))
    return [(output[:-n_val,:-1],output[:-n_val,-1]),(output[-n_val:,:-1],output[-n_val:,-1])]

In [3]:
data = gen_and_load(100000,1000)


In [4]:
train_x ,train_y = data[0]
val_x, val_y = data[1]

In [5]:
opt = keras.optimizers.Adam(learning_rate= 0.005,amsgrad= True)
model = models.Sequential()
model.add(layers.Dense(512,activation= 'relu',input_shape = (train_x.shape[1],),kernel_initializer= 'he_normal'))
model.add(layers.Dense(512,activation= 'relu',kernel_initializer= 'he_normal'))
model.add(layers.Dense(512,activation= 'relu',kernel_initializer= 'he_normal'))
model.add(layers.Dense(1))
model.compile(optimizer = opt,loss = 'mape', metrics = [['mean_absolute_error'],['mean_absolute_percentage_error']])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total params: 534,529
Trainable params: 534,529
Non-trainable params: 0
_________________________________________________________________


2021-11-29 10:53:19.046998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-29 10:53:19.107350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-29 10:53:19.107697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-29 10:53:19.108369: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
def transform_test(fn):
    model.fit(train_x,fn(train_y),validation_data=(val_x,fn(val_y)),batch_size= 256 , epochs = 100, verbose = 0)
    return np.min(model.history.history['val_mean_absolute_percentage_error'])

In [7]:
def f0(array):
    return array
def f1(array):
    return abs(array)**0.25
def f2(array):
    return abs(array)**0.25*10
def f3(array):
    return abs(array)**0.25*100

In [8]:
repeats = 10
history = np.zeros((4,repeats))
for i in range(repeats):
    history[0,i] = transform_test(f0)
    history[1,i] = transform_test(f1)*4
    history[2,i] = transform_test(f2)*4
    history[3,i] = transform_test(f3)*4

  

2021-11-29 10:53:20.019003: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [9]:
history
means = np.mean(history, axis = 0)
std = np.std(history,axis= 0)


In [10]:
print(means , std)

[  3.71812266   8.72344828  60.54226494  74.48291111  88.26468754
  96.74550629 102.79783249 107.3862772  110.8656559  113.58059788] [2.97799571e-01 5.16030076e-01 1.87443022e-02 2.41548715e+01
 4.80238219e+01 6.27127064e+01 7.31944886e+01 8.11419672e+01
 8.71678089e+01 9.18697351e+01]
